# Point Process

In [1]:
import pandas as pd
import numpy as np
import theano
import pymc3 as pm
from scipy.stats import ks_2samp
import seaborn as sns
import matplotlib.pyplot as plt
import glob

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
import warnings

Running on PyMC3 v3.8


## Dirichlet Mixture Model

In [2]:
def Bayesian_mixture_model(ISI_data):
    
    with pm.Model() as model:
        
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.3)
        sigma = pm.Uniform('sigma',lower=0.0001,upper=0.5)
        obs2 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu2,sigma=sigma)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu3,sigma=sigma3)


        w = pm.Dirichlet('w', a=np.array([1., 1., 1.]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)


        #step = pm.Metropolis(target_accept=0.9)
        #trace = pm.sample(step=step,draws=500,tune=100,progressbar=False,random_seed=10,cores=6)
     

    map_estimate = pm.find_MAP(model=model)
    del map_estimate['w_stickbreaking__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    del map_estimate['w']
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    #with model:
    #    ppc_trace = pm.sample_posterior_predictive(trace, 50,model=model,progressbar=False,random_seed=10)
    
    #lista_samples=[]
    #for i in list(ppc_trace['like']):
    #    lista_samples.extend(i)
    
    
    #print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Inverse Gaussian

In [3]:
def Inverse_Gaussian_model(ISI_data):    
    
    with pm.Model() as model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)


        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,tune=1000,cores=6)

        map_estimate = pm.find_MAP(model=model)
        
    
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    
    
    
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 200,model=model)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Import all neural net

In [4]:
list_dir=glob.glob("Data after SS/*.txt")

In [5]:
for net in list_dir:
    if 'healthy' in net:
        print(net,'ok')

Data after SS/2018-10-12T15-33-52MIP nr 5, baseline,control, H9N healthy.h5.txt ok
Data after SS/2019-01-23T11-19-05PoP1 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-30-29MEA2 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-41-41Pop1 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-05-52MIP3 healthy cortical .h5.txt ok
Data after SS/2018-10-12T14-03-45MIP nr 2, baseline,  control, H9N healthy.h5.txt ok


## Run the model on the interval 100-300s

In [ ]:
dataframe = pd.DataFrame()
for net in list_dir:
    print(net)
    list_neurons = np.genfromtxt(net, delimiter=',')
    
    for neuron in list_neurons:
        if neuron[neuron!=0].shape[0]>1000:
            neuron = neuron[neuron!=0]
        else:
            pass
        neuron=neuron[neuron>100*10000]
        neuron=neuron[neuron<300*10000]
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'healthy' in net:
            map_estimate['Target']=0
        elif 'health' in net:
            map_estimate['Target']=0
        else:
            map_estimate['Target']=1
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)


Data after SS/2018-10-12T15-33-52MIP nr 5, baseline,control, H9N healthy.h5.txt


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '8812' (I am process '1709')
/home/riccardo_levi_25/anaconda3/lib/python3.7/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
logp = 622.18, ||grad|| = 0.026819: 100%|██████████| 57/57 [00:00<00:00, 307.30it/s] 
logp = 31,562, ||grad|| = 0.40653: 100%|██████████| 44/44 [00:00<00:00, 339.00it/s]  
logp = -303.97, ||grad|| = 0.00015826: 100%|██████████| 54/54 [00:00<00:00, 1549.32it/s]  
logp = -92.467, ||grad|| = 0.0030395: 100%|██████████| 54/54 [00:00<00:00, 1114.68it/s]  
logp = 9,987.3, ||grad|| = 0.15962: 100%|██████████| 97/97 [00:00<00:00, 578.98it/s]  
logp = -185.7, ||grad|| = 0.01672: 100%|██████████| 51/51 [00:00<00:00, 1129.94it/s]   
logp = 517.23, ||grad|| = 0.042901: 100%|██████████| 74/74 [00:

In [ ]:
final = dataframe.T
final.to_csv('Data after PP/Data')